# Concept portfolio
-------------------------------
This Jupyter notebook contains code and explanatory text for three key visualization products:
* An interactive map
* A time profile plotting tool
* A GIS / Google Earth data export utility

The netCDF filename and other plotting parameters specified below are used globally between all three products. The code cells for the three products are independent of one another, however, so once you specify the global parameters you can choose to run any or all of the following cells. To contextualize the code presented, I provide a brief overview of each of the Python packages used in this notebook (and by the repository as a whole) as a preface.

## Contents
----------------
1. [Overview of tools](#tools)
2. [Global visualization specifications](#global)

<a id="tools"></a>
## 1. Overview of tools
-------------------------------
The list of Python packages here corresponds to those specified in [`environment.yml`](environment.yml), which is the file used to build the [Binder](https://mybinder.org/v2/gh/liamtoney/ashfall_visual/master) for this repository. I've spent quite a bit of time searching through various package options &mdash; for example, there are approximately one gazillion KML writing packages out there &mdash; and this list is the distilled result of my efforts.

### "Standard" tools
- [**matplotlib**](https://matplotlib.org/)
- [**numpy**](http://www.numpy.org/)

### Gridded data I/O
- [**netcdf4**](http://unidata.github.io/netcdf4-python/) &mdash; Required by xarray to read netCDF files.


- [**xarray**](https://xarray.pydata.org/en/latest/) &mdash; Excellent package for working with N-D arrays. Reading a netCDF file with xarray creates an `xarray.Dataset` object, which is essentially an in-memory representation of the netCDF file.

### Georeferenced data I/O
- [**fiona**](http://toblerity.org/fiona/) &mdash; Python wrapper for the **vector** portion of the GDAL library. Can be used to create georeferenced point, line, and polygon information (shapefiles) for ingestion into GIS programs such as ArcMap.


- [**gdal**](https://pypi.org/project/GDAL/) &mdash; Required by both Rasterio and Fiona. It's possible to use this library directly with (for example)
```python
from osgeo import gdal
from osgeo import ogr
```
However, Rasterio and Fiona provide a much friendlier interface.


- [**rasterio**](https://rasterio.readthedocs.io/en/latest/) &mdash; Python wrapper for the **raster** portion of the GDAL library. Can be used to create georeferenced GeoTIFF files for ingestion into GIS programs such as [ArcMap](http://desktop.arcgis.com/en/arcmap/).


- [**simplekml**](https://simplekml.readthedocs.io/en/latest/) &mdash; Provides a simple Python interface for constructing KML and KMZ files for ingestion into programs such as [Google Earth Pro](https://www.google.com/earth/download/gep/agree.html).

### Plotting
- [**bokeh**](https://bokeh.pydata.org/en/latest/) &mdash; A plotting backend alternative to Matplotlib. Offers easily configurable interactivity with some pretty serious features available if you're willing to code your own JavaScript callbacks. Limited support for geographic data, but GeoViews mostly solves this issue.


- [**cartopy**](http://scitools.org.uk/cartopy/) &mdash; I like to think of cartopy (in combination with Matplotlib) as a fairly solid Python substitute for GMT. Cartopy supports a large number of projections and allows for easy translation of data between projections by defining source and destination [`cartopy.crs`](http://scitools.org.uk/cartopy/docs/v0.15/crs/index.html) instances.


- [**geoviews**](http://geo.holoviews.org/) &mdash; GeoViews is the geospatially-oriented version of HoloViews. Like HoloViews, it is compatible with both the Bokeh and Matplotlib backends. The notebooks in [`experiments/`](experiments) explore these various backends. GeoViews has great integration with xarray and cartopy, and at the time of writing is rapidly accumulating powerful new features including reprojection and regridding routines. Check out the [gallery](http://geo.holoviews.org/gallery/index.html) for inspiration.


- [**holoviews**](http://holoviews.org/) &mdash; HoloViews is designed to make visualizing data as trivial as possible. For our purposes we mostly interact with HoloViews `Elements` that are wrapped into geospatially-friendly `GeoElements` by GeoViews. HoloViews works with either Bokeh or Matplotlib as the plotting backend.

### Miscellaneous
- [**colorcet**](https://bokeh.github.io/colorcet/) &mdash; Perceptually uniform colormaps compatible with both Matplotlib and Bokeh.


- [**esmpy**](http://www.earthsystemmodeling.org/esmf_releases/public/ESMF_7_1_0r/esmpy_doc/html/index.html) &mdash; Python interface to the Earth System Modeling Framework (ESMF) regridding utility. Required by xESMF.


- [**geopy**](https://geopy.readthedocs.io/en/latest/) &mdash; Provides "geocoding" functionality. Enter in a location string, such as `'Taupo, NZ'` or `'Te Papa'` or even something as vague as `'steepest street in the world'`, and retrieve the coordinates and address information.


- [**pyproj**](https://jswhit.github.io/pyproj/) &mdash; I mostly use this for transforming coordinates between different coodinate systems using [`pyproj.transform`](https://jswhit.github.io/pyproj/pyproj-module.html#transform).


- [**shapely**](https://shapely.readthedocs.io/en/latest/) &mdash; A powerful library for manipulating 2D geometries. There are many applications of Shapely, but in particular it allows for the packaging of coordinates into geometry objects that can be processed GIS-style. Plays well with Fiona to read/write/process shapefiles.


- [**xesmf**](http://xesmf.readthedocs.io/en/latest/) &mdash; Very slick package for regridding datasets (i.e. reduce the cell size of an ash deposition model).

<a id="global"></a>
## 2. Global visualization specifications
------------------------------------------------------
Specify a `FILENAME` and colorbar limits expressed as `ASH_MIN` and `ASH_MAX`. For plots with colorbars, ash thickness values below `ASH_MIN` and above `ASH_MAX` will be mapped to the colorbar's lowest and highest colors, respectively. This is useful for ignoring trace ash amounts. `CMAP` must be a string corresponding to one of the many excellent palettes included with colorcet. https://bokeh.github.io/colorcet/

In [ ]:
# filename (including path) of HYSPLIT model
FILENAME = '18042918_taupo_15.0_0.01.nc'

# colorbar cutoff values
ASH_MIN = 10**-1
ASH_MAX = 10**2

CMAP = 'linear_kry_5_98_c75'

list(reversed(cc.palette[CMAP]))
cc.cm[CMAP+'_r']

# 1. Reading the HYSPLIT output file
We start by reading the chosen HYSPLIT netCDF file using [xarray](http://xarray.pydata.org/en/stable/), returning an `xarray.Dataset` object. The `Dataset` object is essentially an in-memory representation of the netCDF file. It's self-describing and it contains all of the attributes, coordinates, units etc. of the source file.

In [ ]:
import xarray as xr

FILENAME = '18042918_taupo_15.0_0.01.nc'

model = xr.open_dataset(FILENAME)

We make a few out-of-the-box modifications to the model:
* The edge of the model space for these HYSPLIT files runs right up to the 180<sup>th</sup> meridian, which causes instabilities in some plotting codes that don't know how to "wrap-around" coordinates. To avoid issues we remove the sliver of model space on this edge.
* We trim the model space to the smallest bounding box encompassing all non-zero ash deposition values at the final model time step. This vastly reduces the number of elements in the model, and we don't lose any information since those discarded elements were all zero anyways.
* Finally, we stick in a trivial empty time step at $t=0$, since most of our plots which incorporate the time dimension make more sense when intialized at the actual onset of the eruption.

# [wish list]

time_profiles
* bar plot instead of line plot
* force time axis tick interval to match model time step
* plot total ashfall extent as a patch instead of a contour
* tweak vocabulary of descriptions
* implement time slider for contours
    
map plots
* use white, labelled contours